In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.4.2


In [2]:
!python -V

Python 3.9.19


In [3]:
import pickle
import pandas as pd

In [4]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

/home/ubuntu/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DictVectorizer from version 1.5.0 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/ubuntu/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LinearRegression from version 1.5.0 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [5]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [13]:
year = 2023
month = 3
taxi_type = "yellow"
input_file = f'https://d37ci6vzurychx.cloudfront.net/trip-data/{taxi_type}_tripdata_{year:04d}-{month:02d}.parquet'
output_file = f'output/{taxi_type}/{taxi_type}_tripdata_{year:04d}-{month:02d}.parquet'
df = read_data(input_file)

In [7]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [20]:
df_result = pd.DataFrame()
#df_result['ride_id'] = df['ride_id']
df_result['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')
#df_result['tpep_pickup_datetime'] = df['tpep_pickup_datetime']
#df_result['PULocationID'] = df['PULocationID']
#df_result['DOLocationID'] = df['DOLocationID']
#df_result['actual_duration'] = df['duration']
df_result['predicted_duration'] = y_pred
#df_result['diff'] = df_result['actual_duration'] - df_result['predicted_duration']
#df_result['model_version'] = RUN_ID

In [22]:
df_result

ride_id  predicted_duration
0              2023/03_0           16.245906
1              2023/03_1           26.134796
2              2023/03_2           11.884264
3              2023/03_3           11.997720
4              2023/03_4           10.234486
...                  ...                 ...
3316211  2023/03_3403761           11.952635
3316212  2023/03_3403762           20.049958
3316213  2023/03_3403763           11.595336
3316214  2023/03_3403764           13.113178
3316215  2023/03_3403765           12.899992

[3316216 rows x 2 columns]

In [21]:
df_result.describe()

predicted_duration
count        3.316216e+06
mean         1.420387e+01
std          6.247490e+00
min         -1.160751e+01
25%          1.096814e+01
50%          1.197354e+01
75%          1.413772e+01
max          7.498748e+01

In [11]:
!mkdir output

In [12]:
!mkdir output/yellow

In [23]:
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)